## Long-Term Memory - Remember Across Conversations

### Store User Preferences That Work Across all Threads

Learning Objectives:
* Understand short term vs long term memory
* Store user preference across threads automatically
* Search Memories with semantic search

### Real-World use cases:
1. **Personal Assistants**: Remember preferences across all chats
2. **Customer Support**: Track customer info across ttickets
3. **E-commerce**: Store shopping preferences
4. **Education**: Track learning progress

### 1. Imports

In [11]:
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END

# For Short-Term Memory Persistence
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.postgres import PostgresSaver

# For Long-Term Memory Persistence
from langgraph.store.postgres import PostgresStore

from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.tools import tool
import psycopg 
import os

### 2. LLM Configuration

In [12]:
BASE_URL="http://localhost:11434"
MODEL_NAME="gemma3"
EMBEDDING_MODEL="nomic-embed-text"

llm = ChatOllama(model=MODEL_NAME, base_url=BASE_URL)

### 3. Store and Saver Setup

In [13]:
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=BASE_URL)

def embed_texts(texts: list[str]) -> list[list[float]]:
    return embeddings.embed_documents(texts)

db_url = os.getenv("POSTGRESQL_URL")

# Chat History Memory Connection
checkpointer_conn = psycopg.connect(db_url, autocommit=True, prepare_threshold=0)
checkpointer = PostgresSaver(checkpointer_conn)

# Memory Store Connection
store_conn = psycopg.connect(db_url, autocommit=True, prepare_threshold=0)
store = PostgresStore(store_conn, index = {'embed': embed_texts, 'dims': 768})

# First Time Setup
checkpointer.setup()
store.setup()

### Tool Usage - Memory Management Tools

### Memory Operations: `put()` , `get()` , `search()` , `delete()`

In [14]:
user_id = "manmath123"
namespace = (user_id, "preferences")

store.put(namespace, 'food', {"diet": "veg", "likes": ["pasta", "pizza", "spaghethi"]})
store.put(namespace, "color", {"favorite": "blue", "dislike": "brown"})
store.put(namespace, "work", {"role": "Data Scientist", "interests": ["machine learning", "AI", "Gen AI", "Agents"]})

### Semantic Search for Memory Retrieval

In [15]:
query = "What does Manmath likes to eat?"
results = store.search(namespace, query=query, limit=1)
results

[Item(namespace=['manmath123', 'preferences'], key='food', value={'diet': 'veg', 'likes': ['pasta', 'pizza', 'spaghethi']}, created_at='2026-01-26T14:01:54.237809+00:00', updated_at='2026-01-26T18:16:06.603708+00:00', score=0.6844177858010962)]

In [16]:
query = "What colors does Manmath like?"
results = store.search(namespace, query=query, limit=1)
results

[Item(namespace=['manmath123', 'preferences'], key='color', value={'dislike': 'brown', 'favorite': 'blue'}, created_at='2026-01-26T14:01:54.576729+00:00', updated_at='2026-01-26T18:16:07.019369+00:00', score=0.7099188757330387)]

### Agent with Automatic Memory Storage

In [17]:
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    user_id: str

In [18]:
# Tools for memory management
# save user memory and get user memory
@tool
def save_user_memory(user_id: str, category:str, information: dict) -> str:
    """
    Save user preference or information to long-term memory.

    Args:
        user_id: User Identifier
        category: Category of information (e.g. 'food', 'work', 'hobbies', 'shcedule', 'location')
        information: Dictionary containing the information to save
    """
    namespace = (user_id, "preferences")
    store.put(namespace, category, information)
    return f"Saved {category} preferences."

In [19]:
@tool
def get_user_memory(user_id: str, category: str) -> str:
    """
    Retrieve user preference or information from long-term memory.

    Args:
        user_id: User Identifier
        category: Category of Information to retrieve (e.g. 'food', 'work', 'hobbies')
    """
    namespace = (user_id, "preferences")
    item = store.get(namespace, category)

    if item:
        return f"{category}: {item.value}"
    else:
        return f"No {vategory} information found"

In [20]:
get_user_memory.invoke({"user_id": "manmath123", "category": "color"})

"color: {'dislike': 'brown', 'favorite': 'blue'}"